In [1]:
import akshare as ak
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)

code_list = ['sh000300', "sz399905"]

In [2]:
index_data = {}
for code in code_list:
    df = ak.stock_zh_index_daily(symbol=code)
    df["date"] = df["date"].apply(lambda x: x.strftime("%Y-%m-%d"))
    df = df[df["date"] >= "2016-01-01"]
    index_data[code] = {"code": code, "data": df}

In [3]:
import talib

In [4]:
for code in code_list:
    j = index_data[code]
    df = j["data"]
    close = df["close"].to_numpy()
    j["close"] = close
    j["open"] = df["open"].to_numpy()
    j["rsi2"] = talib.RSI(close, timeperiod=2)

In [5]:
dates = index_data["sh000300"]["data"]["date"].to_numpy()

In [10]:
# 单独测试RSI
account = 1000000.0
profits = []
r_stop = 75
cur_code = ""
amount = 0
prev_account = 0
for (index, date) in enumerate(dates):
    for code in code_list:
        # 昨天收盘RSI2满足条件，第二天开盘开干
        rsi2 = index_data[code]["rsi2"][index]
        close = index_data[code]["close"][index]
        if rsi2 < 5 and cur_code == "":
            cur_code = code
            amount = account / close
            prev_account = account
            account = 0
            print (date, "buy ", code)
        if rsi2 >= r_stop and cur_code == code:
            account = amount * close
            cur_code = ""
            amount = 0
            profit = account - prev_account
            print (date, "sell ", code, profit)
            profits.append(profit)

2016-01-28 buy  sh000300
2016-02-04 sell  sh000300 45905.816755181644
2016-02-29 buy  sz399905
2016-03-02 sell  sz399905 78722.7568994863
2016-04-21 buy  sz399905
2016-05-03 sell  sz399905 37063.67441695533
2016-05-09 buy  sh000300
2016-05-23 sell  sh000300 8188.192612683633
2016-06-13 buy  sh000300
2016-06-22 sell  sh000300 25798.60341660399
2016-08-26 buy  sh000300
2016-08-31 sell  sh000300 7485.535298121627
2016-09-12 buy  sh000300
2016-09-22 sell  sh000300 10516.337352226255
2016-12-12 buy  sz399905
2016-12-21 sell  sz399905 19135.674182507675
2017-01-13 buy  sz399905
2017-01-23 sell  sz399905 -4475.998182093725
2017-03-30 buy  sz399905
2017-04-05 sell  sz399905 31638.746603198815
2017-04-18 buy  sh000300
2017-05-12 sell  sh000300 -28150.238585378742
2017-05-23 buy  sz399905
2017-06-06 sell  sz399905 16377.803284351481
2017-07-17 buy  sz399905
2017-07-20 sell  sz399905 47329.812629646156
2017-08-11 buy  sh000300
2017-08-17 sell  sh000300 26258.829795402242
2017-10-19 buy  sz399905


In [11]:
print ('total account = ', account)
print('total len = ', len(profits))
winning_len = len(list(filter(lambda x: x > 0, profits)))
print('winning len = ', winning_len)
print("winning rate = ", winning_len * 1.0 / len(profits))
print("avg profit = ", sum(profits) / len(profits))
print("max loss = ", min(profits))

total account =  1784105.5440679227
total len =  83
winning len =  60
winning rate =  0.7228915662650602
avg profit =  9447.054747806298
max loss =  -118679.74805991584


In [20]:
# 累积RSI
account = 1000000.0
profits = []
r_stop = 65
cur_code = ""
amount = 0
prev_account = 0
for (index, date) in enumerate(dates):
    for code in code_list:
        # 昨天收盘RSI2满足条件，第二天开盘开干
        rsi2 = index_data[code]["rsi2"][index]
        cu_rsi = rsi2 + index_data[code]["rsi2"][index-1]
        close = index_data[code]["close"][index]
        if cu_rsi < 10 and cur_code == "":
            cur_code = code
            amount = account / close
            prev_account = account
            account = 0
            print (date, "buy ", code)
        if rsi2 >= r_stop and cur_code == code:
            account = amount * close
            cur_code = ""
            amount = 0
            profit = account - prev_account
            print (date, "sell ", code, profit)
            profits.append(profit)

2016-09-13 buy  sh000300
2016-09-21 sell  sh000300 1933.2381078863982
2016-12-15 buy  sh000300
2017-01-03 sell  sh000300 537.7942794042174
2017-01-13 buy  sz399905
2017-01-20 sell  sz399905 -11758.283143674606
2017-04-19 buy  sh000300
2017-05-12 sell  sh000300 -17393.298305275966
2017-12-01 buy  sh000300
2017-12-05 sell  sh000300 10232.649977049674
2018-02-01 buy  sz399905
2018-02-14 sell  sz399905 -37783.99172184279
2018-03-23 buy  sh000300
2018-04-10 sell  sh000300 5385.759316961048
2018-04-17 buy  sh000300
2018-04-19 sell  sh000300 16036.432082346873
2018-05-29 buy  sz399905
2018-06-05 sell  sz399905 -9342.659132449655
2018-06-19 buy  sz399905
2018-06-29 sell  sz399905 14168.266304310644
2018-08-02 buy  sh000300
2018-08-07 sell  sh000300 -602.0749043485848
2018-08-17 buy  sh000300
2018-08-21 sell  sh000300 29184.64487729466
2018-09-11 buy  sz399905
2018-09-19 sell  sz399905 7231.989019714179
2018-10-12 buy  sz399905
2018-10-22 sell  sz399905 12982.955264040502
2018-11-26 buy  sz3999

In [21]:
print ('total account = ', account)
print('total len = ', len(profits))
winning_len = len(list(filter(lambda x: x > 0, profits)))
print('winning len = ', winning_len)
print("winning rate = ", winning_len * 1.0 / len(profits))
print("avg profit = ", sum(profits) / len(profits))
print("max loss = ", min(profits))

total account =  1708990.252299984
total len =  59
winning len =  46
winning rate =  0.7796610169491526
avg profit =  12016.783937287864
max loss =  -55064.914274730254
